In [1]:
import numpy as np
import pandas as pd
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
train = pd.read_csv('data/creditdefault_train.csv')
test = pd.read_csv('data/creditdefault_test.csv')

In [5]:
train

,Y,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23
0,1,20000,2,2,1,24,2,2,-1,-1,...,689,0,0,0,0,689,0,0,0,0
1,0,50000,2,2,1,37,0,0,0,0,...,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000
2,0,50000,1,2,1,57,-1,0,-1,0,...,35835,20940,19146,19131,2000,36681,10000,9000,689,679
3,0,50000,1,1,2,37,0,0,0,0,...,57608,19394,19619,20024,2500,1815,657,1000,1000,800
4,0,500000,1,1,2,29,0,0,0,0,...,445007,542653,483003,473944,55000,40000,38000,20239,13750,13770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0,100000,1,1,2,38,0,-1,-1,0,...,102996,70626,69473,55004,2000,111784,4000,3000,2000,2000
14996,0,220000,1,3,1,39,0,0,0,0,...,208365,88004,31237,15980,8500,20000,5003,3047,5000,1000
14997,0,150000,1,3,2,43,-1,-1,-1,-1,...,3502,8979,5190,0,1837,3526,8998,129,0,0
14998,1,30000,1,2,2,37,4,3,2,-1,...,2758,20878,20582,19357,0,0,22000,4200,2000,3100


In [110]:
test

,Y,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23
0,1,120000,2,2,2,26,-1,2,0,0,...,2682,3272,3455,3261,0,1000,1000,1000,0,2000
1,0,90000,2,2,2,34,0,0,0,0,...,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000
2,0,140000,2,3,1,28,0,0,2,0,...,12108,12211,11793,3719,3329,0,432,1000,1000,1000
3,0,20000,1,3,2,35,-2,-2,-2,-2,...,0,0,13007,13912,0,0,0,13007,1122,0
4,0,200000,2,3,2,34,0,0,2,0,...,5535,2513,1828,3731,2306,12,50,300,3738,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0,150000,1,1,2,35,-1,-1,-1,-1,...,-3,780,0,0,9054,0,783,0,0,0
14996,0,140000,1,2,1,41,0,0,0,0,...,139110,138262,49675,46121,6000,7000,4228,1505,2000,2000
14997,0,10000,1,3,1,43,0,0,0,-2,...,0,0,0,0,2000,0,0,0,0,0
14998,1,80000,1,2,2,34,2,2,2,2,...,79384,77519,82607,81158,7000,3500,0,7000,0,4000


In [111]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self,dataframe):
        self.dataframe = dataframe
        
    def __getitem__(self,index):
        row = torch.from_numpy(self.dataframe.iloc[index].to_numpy())
        features = row[1:].to(torch.float32)
        label = row[0]
        '''
        label = torch.zeros(2)
        label[int(row[0])] = 1
        '''
        return features,label.to(torch.float32)
        
    def __len__(self):
        return len(self.dataframe)
        

In [112]:
batchSize = 100
trainData = CustomDataset(dataframe = train)
trainLoader = DataLoader(trainData, batch_size = batchSize, shuffle = True)

In [113]:
sampler = iter(trainLoader)
print(next(sampler))

[tensor([[5.0000e+04, 1.0000e+00, 3.0000e+00,  ..., 3.0000e+01, 1.0310e+03,
         2.0201e+04],
        [2.0000e+05, 1.0000e+00, 2.0000e+00,  ..., 3.1000e+03, 4.4000e+03,
         2.9300e+03],
        [2.6000e+05, 1.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [4.2000e+05, 1.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.7000e+05, 1.0000e+00, 2.0000e+00,  ..., 0.0000e+00, 1.0000e+03,
         1.0300e+03],
        [2.2000e+05, 2.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         6.0000e+03]]), tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
  

In [257]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        self.fc1 = nn.Linear(23,75)
        self.fc2 = nn.Linear(75,75)
        self.fc3 = nn.Linear(75,8)
        self.fc4 = nn.Linear(8,2)
        
        self.bn1 = nn.BatchNorm1d(75)
        self.bn2 = nn.BatchNorm1d(8)
        self.bn3 = nn.BatchNorm1d(75)
        self.drop1 = nn.Dropout(p=0.1)
        
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.bn3(x)
        x = F.relu(self.fc2(x))
        x = self.bn1(x)
        x = F.relu(self.fc3(x))
        x = self.bn2(x)
        #x = self.drop1(x)
        x = self.fc4(x)
        
        return x

In [258]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimiser = optim.Adam(net.parameters(), lr=1e-5)

In [259]:
for epoch in range(10):
    runningLoss = 0.0
    
    for i, data in enumerate(trainLoader,0):
        
        inputs, labels = data
        #labels = labels.unsqueeze(1)
        optimiser.zero_grad()
                
        outputs = net(inputs)
        #_,predicted = torch.max(outputs,1)
        #print(outputs,outputs.shape,labels,labels.shape)

        loss = criterion(outputs, labels.long())
        loss.backward()
        optimiser.step()
        
        runningLoss += loss.item()
        
    #print(outputs,torch.max(outputs,1),labels)
    print(f'[{epoch + 1}] loss: {runningLoss / i:.3f}')
    runningLoss = 0.0

[1] loss: 0.852
[2] loss: 0.831
[3] loss: 0.813
[4] loss: 0.803
[5] loss: 0.792
[6] loss: 0.784
[7] loss: 0.778
[8] loss: 0.770
[9] loss: 0.762
[10] loss: 0.759


In [123]:
testData = CustomDataset(dataframe = test)
testLoader = DataLoader(testData, batch_size = batchSize, shuffle = True)

In [125]:
testSample = iter(testLoader)

In [142]:
next(testSample)

[tensor([[3.0000e+04, 2.0000e+00, 3.0000e+00,  ..., 1.1640e+03, 1.0270e+03,
          1.5000e+03],
         [1.0000e+04, 2.0000e+00, 2.0000e+00,  ..., 7.7800e+02, 0.0000e+00,
          1.5800e+02],
         [2.4000e+05, 2.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 3.8000e+02,
          7.6000e+02],
         ...,
         [7.0000e+04, 1.0000e+00, 2.0000e+00,  ..., 5.0000e+03, 3.0000e+02,
          4.2200e+03],
         [1.5000e+05, 2.0000e+00, 2.0000e+00,  ..., 9.6200e+02, 8.2100e+02,
          3.8000e+02],
         [1.5000e+05, 2.0000e+00, 3.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00]]),
 tensor([0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0.,
         1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [261]:
correct, total = 0, 0
posFound, totalPos = 0, 0

with torch.no_grad():
    for data in testLoader:
        inputs, labels = data
        outputs = net(inputs)
        
        #print('o',outputs)
        #print('r',outputs.round())
        #print('l',labels)
        
        _,predicted = torch.max(outputs,1)
        '''
        print()
        print('p',predicted)
        print('l',labels)
        print('s',torch.softmax(outputs,dim=1))
        '''
        for label, prediction in zip(labels,predicted):
            print(label,prediction)
            if label == prediction: correct += 1
            total += 1
            if label == 1:
                if prediction == 1: posFound += 1
                totalPos += 1
        print(correct,total,correct/total)
        print(posFound,totalPos,posFound/totalPos)
            
print(f'Accuracy on creditdefault dataset: {100*(correct/total)}%')
print(f'Delinquency Detection Rate: {100*(posFound/totalPos)}%')

tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(1.) tensor(0)
tensor(1.) tensor(0)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(1.) tensor(0)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) te

tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(1.) tensor(1)
tensor(1.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) te

tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(0)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(0)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(0)
tensor(1.) tensor(0)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) te

tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
981 2200 0.4459090909090909
387 482 0.8029045643153527
tensor(0.) tensor(1)
tensor(1.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(0)
tensor(1.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(0)
tensor(1.) tensor(1)
tensor(1.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(1.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)


tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) te

...

tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(1.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
5338 11900 0.44857142857142857
2089 2657 0.7862250658637561
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tenso

tensor(1.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(1.) tensor(0)
tensor(0.) tensor(1)
tensor(1.) tensor(0)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
5606 12500 0.44848
2186 2780 0.7863309352517985
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(1.) tensor(1)
tensor(

tensor(1.) tensor(1)
tensor(1.) tensor(1)
tensor(1.) tensor(1)
tensor(1.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(1.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(1.) te

tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(1.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
6233 13900 0.44841726618705036
2447 3098 0.7898644286636539
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(1.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(1.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tenso

tensor(1.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(0)
tensor(1.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(1.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(1.) tensor(0)
tensor(0.) tensor(1)
tensor(1.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(0)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) tensor(1)
tensor(0.) te

# Conclusion

This model has less than 50% accuracy, however, it detects 79% of deliquencies on the testing dataset; changing the parameters gave us 82% accuracy but the model was only able to detect less thaan 10% of delliquencies. This model sacrifices overall accuracy for better detection rates.